In [1]:
region_name = "MDIS_10-800" #"TDS" # _ELAISN1
probabilities = [0.000063342484]  # 3 sigma 0.002699796063, 4 sigma 0.000063342484
srcs_ids = [15179]  #RR*s [2052 ,15179 ,15934] # TDS RR*
srcs_ids = [900 ,11630 ,52486 ,53151 ,58506 ,72801 ,82069 ,84614 ,163052 ,165195 ,179997 ,] # MDIS WD

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord
from astropy.timeseries import LombScargle,TimeSeries,BinnedTimeSeries
from astropy.units import cds
from astropy.table import QTable
from astropy.time import Time

#cds.enable()  
import numpy as np

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux, sel_sources, dd_filter2id

region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
rg = Region()
rg.load_from_fits(region_fname)
#rg.remove_table("tt_coadd_detections")
#rg.tt_coadd_sources.remove_columns(["nr_det","obs_filter_id","flux_nxv","flux_var","flux_cpval","flux_rchiq","fd_src_id","assoc_id","assoc_dist","assoc_ffactor","assoc_fdiff_s2n","rg_fd_id","nr_fd_dets","pos_xv","pos_var","pos_cpval","pos_rchiq"])
#rg.tt_sources.remove_columns(["fd_src_id","rg_fd_id"])
#rg.write_to_fits(region_fname)
rg.tt_sources.add_index("rg_src_id")
src_idx = rg.tt_sources.loc_indices[srcs_ids]
tt_src = rg.tt_sources[src_idx]
display(tt_src)

nr_det,ra,dec,pos_err,pos_xv,pos_var,pos_cpval,pos_rchiq,assoc_id,assoc_dist,obs_filter_id,sel,flux,flux_err,flux_nxv,flux_var,flux_cpval,flux_rchiq,assoc_ffactor,assoc_fdiff_s2n,rg_src_id,nr_fd_srcs,hr,hr_err
,deg,deg,arcsec,arcsec2,arcsec2,,,,arcsec,,,1e-06 Jy,1e-06 Jy,,1e-12 Jy2,,,,,,,,
int32[2],float64,float64,float32,float32,float32,float32,float32,int64,float32,int32[2],bool,float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],int32,int32,float32,float32
35 .. 55,215.3258439968967,52.596545351520945,0.061777495,-0.073819295,0.27413565,0.076620735,0.8073005,2996,0.15070449,1 .. 2,True,69.76633 .. 78.84212,0.2989939 .. 0.35905793,0.0028636206 .. 0.00013604689,17.501484 .. 8.515412,3.2471413e-13 .. 0.18849026,3.8351877 .. 1.1662406,0.91958094 .. 0.98226017,-18.350433 .. -3.4928458,900,6,1.1639541,0.017203152
67 .. 46,149.76167436162893,1.6465905160899403,0.050711945,-0.121467255,0.17606305,0.9983214,0.60001004,51881,0.11816379,1 .. 2,True,154.65216 .. 112.11728,0.3336962 .. 0.5372641,0.00080672395 .. 0.00011547043,31.759632 .. 23.498913,1.1646499e-18 .. 0.1507282,3.3541517 .. 1.2172611,1.0050958 .. 0.9974542,1.9505055 .. -0.43859452,11630,4,0.72394663,0.005090711
62 .. 41,333.74330229428716,-18.1284418039241,0.07469829,-0.26511413,0.374089,0.9973886,0.6041299,227436,0.26807556,1 .. 2,True,15.743377 .. 4.2345347,0.17000161 .. 0.11394305,0.01079712 .. 0.052266393,4.8854165 .. 1.7236115,4.6363047e-07 .. 0.000100011224,2.1521604 .. 2.0515473,0.9985297 .. 0.91966665,-0.10560306 .. -2.440815,52486,2,0.2847042,0.013879122
32 .. 4,333.54135905569626,0.8794295342456827,0.106176615,-0.26969245,0.14336883,0.9997828,0.34498087,230050,0.11125645,1 .. 2,True,20.118876 .. 4.6691704,0.19772485 .. 0.3908768,0.056504067 .. -0.026277466,24.190218 .. 0.044125017,0.0 .. 0.97339696,18.964977 .. 0.075116284,0.9191351 .. 1.0333134,-6.522281 .. 0.2772817,53151,2,0.29695582,0.026712412
25 .. 12,261.0003596219375,60.747724074915844,0.09249079,-0.12485813,0.21191294,0.78836334,0.5457976,254894,0.072778955,1 .. 2,True,42.035763 .. 49.616528,0.2805657 .. 0.656898,0.020497339 .. 0.0150321275,38.25497 .. 67.96018,0.0 .. 2.7350098e-33,21.373968 .. 16.616245,0.97845787 .. 0.93842715,-2.4446874 .. -4.0401735,58506,2,1.0894871,0.017481763
15 .. -1,244.14364475974057,54.1699250866851,0.14302015,-0.149046,0.15964259,0.6340721,0.52604604,83326,0.20260328,1 .. 0,True,46.74693 .. -1.0,0.40762272 .. -1.0,0.020200865 .. -100.0,46.770695 .. -1.0,0.0 .. -1.0,19.716309 .. -1.0,0.98185104 .. -100.0,-1.8601466 .. -10000.0,72801,2,-1.0,-1.0
21 .. 1,219.2285419319909,34.06670048559544,0.11800814,-0.10925614,0.2025599,0.68355554,0.61700284,375016,0.035019293,1 .. 2,True,29.51347 .. 35.736923,0.35198805 .. 2.8622541,0.013819349 .. -100.0,19.32615 .. -100.0,5.677633e-20 .. -1.0,6.986842 .. -1.0,1.0026231 .. 0.9697317,0.18479453 .. -0.3386549,82069,2,1.1706479,0.11864556
18 .. 8,218.05243811169714,35.84090332817147,0.12735511,-0.29414088,0.17379755,0.96632844,0.39901668,386808,0.07959464,1 .. 2,True,14.612254 .. 6.05339,0.2779116 .. 0.37645307,0.030357704 .. 0.055571366,9.390714 .. 3.3930244,1.0763691e-15 .. 0.020767232,6.486113 .. 2.36001,0.92887527 .. 0.9908855,-3.061352 .. -0.11080818,84614,2,0.3186552,0.026751982


In [3]:
dd_lcs = rg.get_light_curve(rg_src_ids=srcs_ids)

def plot_LombScargle(rg_src_id):
    fig, axs = plt.subplots(1, 2, figsize=(14, 6), num="LombScargle", clear=True)
    axs = axs.flatten()
    plt.sca(axs[0])
    lc = dd_lcs[rg_src_id][dd_lcs[rg_src_id]["obs_filter_id"]==1]
    lc["time"]=Time(lc["time"].quantity, format="mjd")
    dt = lc["time"][1:]-lc["time"][0:-1]
    f_max = np.max(1/dt)/2

    f_min = 2/ (np.max(lc["time"]) - np.min(lc["time"]))
    print(f_min,f_max)
    
    ts = TimeSeries(data=lc)
    
    ls = LombScargle(lc["time"], lc["flux"],lc["flux_err"])

    #Frquency vs power plot with confidence itnervals
    freq,power = ls.autopower(minimum_frequency=f_min,maximum_frequency=f_max) #
    conf = ls.false_alarm_level(probabilities)
    
    plt.axhline(conf[0],linewidth=0.5, ls="--", color='k')
    plt.plot(freq, power)
    plt.xscale('log')
    plt.xlabel('Frequency [1/day]')
    plt.ylabel('LS power')
    plt.show()

    p_peak = power.max()
    idx_pmax = np.argmax(power) #np.where(power == p_peak)
    f_peak = freq[idx_pmax]
    Pval =ls.false_alarm_probability(p_peak)
    
    #Show best fit model
    plt.sca(axs[1])
    ts_folded = ts.fold(period=1/f_peak, epoch_time=lc["time"][0])
    plt.plot(ts_folded.time.jd, ts_folded['flux'], 'k.', markersize=4)
    plt.xlabel('Time (days)')
    plt.ylabel('Flux')
    
    #time_fit = np.linspace(0, 1)*uu.d
    #flux_fit = ls.model(time_fit*uu.d, best_freq)
    #plt.sca(axs[1])
    #plt.plot(time_fit,flux_fit,"ob")
    #plt.show()
    
    fig = plt.figure(figsize=(14, 6))
    vvis.plot_light_curve(rg,rg_src_ids = rg_src_id, fig=fig)
    plt.show()

    print(f"Tested {len(freq)} frequencies, maximum power at frequency {f_peak }, period {1/f_peak}, whith a false probability of {Pval}")
widgets.interactive(plot_LombScargle, rg_src_id=srcs_ids)

interactive(children=(Dropdown(description='rg_src_id', options=(900, 11630, 52486, 53151, 58506, 72801, 82069…

In [4]:
24*60/90 # MAximum Nr of orbits per day

16.0

In [5]:
1/(3600/4)

0.0011111111111111111